In [1]:
# Preparando o ambiente no Google Drive:

# Importando a biblioteca
from google.colab import drive

# Isso irá pedir sua autorização
drive.mount('/content/drive')

# Agora, seu Drive estará disponível em: /content/drive/My Drive

Mounted at /content/drive


# Mapas Emocionais


## Importação das bibliotecas


In [2]:
import sys
# Biblioteca para mapa
import folium
from folium import plugins

# Biblioteca para conexao com o banco de dados
!pip install psycopg2-binary
import psycopg2
psycopg2.extensions.register_type(psycopg2.extensions.UNICODE)
psycopg2.extensions.register_type(psycopg2.extensions.UNICODEARRAY)

# Biblioteca para os dados em geojson
import json

# Bibliotecas dos widgets
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import AppLayout
from ipywidgets import Layout, VBox, Label

# Biblioteca de mediacao dos parametros de conexao do banco de dados
from configparser import ConfigParser


     |████████████████████████████████| 3.0MB 5.0MB/s 


## Conexão com o banco de dados

In [3]:
def config(filename='/content/drive/My Drive/Colab Notebooks/database_emoc.ini', section='postgresql'):
    # create a parser
    parser = ConfigParser()
    # read config file
    parser.read(filename)

    # get section, default to postgresql
    db = {}
    if parser.has_section(section):
        params = parser.items(section)
        for param in params:
            db[param[0]] = param[1]
    else:
        raise Exception('Section {0} not found in the {1} file'.format(section, filename))

    return db

In [4]:
conn = None
try:
  # read connection parameters
  params = config()
  # connect to the PostgreSQL server
  print('Connecting to the PostgreSQL database...')
  conn = psycopg2.connect(**params)
  # create a cursor
  cursor = conn.cursor()
  print ('Conexao realizada com sucesso')
except (Exception, psycopg2.DatabaseError) as error:
  print(error)
  print ('Erro na conexao com o banco de dados')


Connecting to the PostgreSQL database...
Conexao realizada com sucesso


## Mapa 1 - Escolha uma emoção e veja em quais ruas ela foi atribuída



In [5]:
# Combox com as emoções
# Funcao para fazer a lista do comboBox 
def lista_emoc():
  emoc = []
  cursor.execute("""SELECT cod_emoc_ruas, tipo FROM emocoes;""")
  fet_list = cursor.fetchall()
  conn.commit()
  for cod_emoc_ruas, tipo in fet_list:
    emoc.append((tipo, cod_emoc_ruas))
  return emoc

# Definindo a caixa de escolha
comboEmoc = widgets.Dropdown(
    options = lista_emoc(),
    description='Emocao:',
)
# Mostrando a caixa de escolha
display(comboEmoc)


Dropdown(description='Emocao:', options=(('raiva odio', 'raiva_odio'), ('cansaco', 'cansaco'), ('tedio', 'tedi…

In [7]:
comboEmoc_esc = comboEmoc.value

# Consulta emocoes geometria linha
cursor.execute("""CREATE OR REPLACE VIEW emoc_selec AS SELECT ruas_emoc_wgs.* FROM ruas_emoc_wgs WHERE %s >= 1; """ %comboEmoc_esc)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features 
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM emoc_selec As lg   ) As f )  As fc;""")
emoc_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Consulta pontos sobre linhas
cursor.execute("""CREATE OR REPLACE VIEW emoc_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM emoc_selec; """)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM emoc_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Consulta emocoes geometria ponto
cursor.execute("""CREATE OR REPLACE VIEW pts_emoc_selec AS SELECT pts_emoc.*, ST_Transform(geom,4326) As geom_wgs FROM pts_emoc WHERE %s >= 1; """ %comboEmoc_esc)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM pts_emoc_selec As lg   ) As f )  As fc;""")
pts_emoc_geojson = json.dumps(cursor.fetchall())
conn.commit()
# cursor.close()
# conn.close()

# Consulta emocoes por GENERO
# Feminino
cursor.execute("""CREATE OR REPLACE VIEW female_emoc_selec AS SELECT trj_user_modal.* FROM trj_user_modal, usuario 
WHERE trj_user_modal.user_id = usuario.user_id AND usuario.genero = 'feminino' AND %s >= 1; """ %comboEmoc_esc)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT ogc_fid) As l)) As properties
FROM female_emoc_selec As lg ) As f )  As fc;""")
female_emoc_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Masculino
cursor.execute("""CREATE OR REPLACE VIEW male_emoc_selec AS SELECT trj_user_modal.* FROM trj_user_modal, usuario 
WHERE trj_user_modal.user_id = usuario.user_id AND usuario.genero = 'masculino' AND %s >= 1; """ %comboEmoc_esc)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT ogc_fid) As l)) As properties
FROM male_emoc_selec As lg ) As f )  As fc;""")
male_emoc_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Consulta de todos os trajetos
cursor.execute("""SELECT ruas_emoc_wgs.* FROM ruas_emoc_wgs; """)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features 
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM ruas_emoc_wgs As lg   ) As f )  As fc;""")
ruas_wgs_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Definindo o mapa
m = folium.Map(location = [-25.44,-49.24],
               zoom_start = 15,
               tiles = 'Cartodb Positron',
               attr = '© contribuidores do OpenStreetMap (CC BY-SA 2.0)')

ft_grp0 = folium.FeatureGroup(name = 'Todos os trajetos')
# Camada das linhas
ruas_wgs = folium.GeoJson(
    ruas_wgs_geojson [2:len(ruas_wgs_geojson )-2],
    name = 'geojson',
    style_function = lambda feature: {
            'color': 'gray',
            'weight': 1,
            },  
    overlay=True
    ).add_to(ft_grp0)

# Definindo um group layer para linhas mais pontos
ft_grp1 = folium.FeatureGroup(name = 'Emocoes linhas')

# Camada das linhas
ruas_emoc = folium.GeoJson(
    emoc_geojson[2:len(emoc_geojson)-2],
    name = 'geojson',
    style_function = lambda feature: {
            'color': 'black',
            'weight': 2,
            },  
    overlay=True
    ).add_to(ft_grp1)

# Camada dos pontos sobre as linhas
gjson_lnstopts = json.loads(lnstopts_geojson[2:len(lnstopts_geojson)-2])

# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = '%s'; """ %comboEmoc_esc)
id_emoji = cursor.fetchall()
conn.commit()
icon_url = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  ft_grp1.add_child(marker)

# Definindo um group layer para os pontos
ft_grp2 = folium.FeatureGroup(name = 'Emocoes pontos')

# Camada dos pontos
gjson_marker = json.loads(pts_emoc_geojson[2:len(pts_emoc_geojson)-2])

# Condicão que teste que a feição tem a geometria ponto para adicionar ao mapa
if gjson_marker['features'] != None:
  # Estilizando os pontos do geojson
  for feature in gjson_marker['features']:
    lon, lat = feature['geometry']['coordinates']
    icon = folium.features.CustomIcon(icon_url,
                                      icon_size=(20, 20))
    marker = folium.Marker([lat, lon], icon=icon)
    ft_grp2.add_child(marker)
    m.add_child(ft_grp2)
else:
  pass

# Camada das emocoes por GENERO
# Definindo um group layer para linhas mais pontos - FEMININO
ft_grp3 = folium.FeatureGroup(name = 'Genero feminino', show = False)

# Camada das linhas - FEMININO
female_ruas_emoc = folium.GeoJson(
    female_emoc_geojson[2:len(female_emoc_geojson)-2],
    name = 'geojson',
    style_function = lambda feature: {
            'color': 'red',
            'weight': 2,
            },  
    overlay=True
    ).add_to(ft_grp3)

# Definindo um group layer para linhas mais pontos - MASCULINO
ft_grp4 = folium.FeatureGroup(name = 'Genero masculino', show = False)

# Camada das linhas - MASCULINO
male_ruas_emoc = folium.GeoJson(
    male_emoc_geojson[2:len(male_emoc_geojson)-2],
    name = 'geojson',
    style_function = lambda feature: {
            'color': 'blue',
            'weight': 3,
            },  
    overlay=True
    ).add_to(ft_grp4)

m.add_child(ft_grp0)
m.add_child(ft_grp1)
m.add_child(ft_grp3)
m.add_child(ft_grp4)

folium.LayerControl().add_to(m)
m

## Mapa 2 - Escolha uma rua e veja quais emoções foram atribuídas a ela

In [ ]:
# Combox com as ruas que fazem parte dos trajetos
# Funcao para fazer a lista do comboBox 
def lista_ruas():
  ruas = []
  cursor.execute("""SELECT nmvia FROM ruas_emoc_wgs GROUP BY nmvia;""")
  fet_list = cursor.fetchall()
  conn.commit()
  for ruas_emoc_wgs in fet_list:
    ruas.append(ruas_emoc_wgs[0])
  return ruas

# Definindo a caixa de escolha
comboRuas = widgets.Dropdown(
    options = lista_ruas(),
    description='Ruas:',
)
# Mostrando a caixa de escolha
display(comboRuas)


RHJvcGRvd24oZGVzY3JpcHRpb249dSdSdWFzOicsIG9wdGlvbnM9KHUnUi4gQ0VMLiBKT1x4YzNceDgzTyBEQSBTSUxWQSBTQU1QQUlPJywgdSdSLiBNQVJJQU5PIFRPUlJFUycsIHUnUi4gUk/igKY=


In [ ]:
nm_rua = comboRuas.value
print(nm_rua)

cursor.execute("""CREATE OR REPLACE VIEW rua_selec AS SELECT ruas_emoc_wgs.* FROM ruas_emoc_wgs WHERE nmvia = '%s'; """ %nm_rua)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
 FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l )) As properties
 FROM rua_selec As lg   ) As f )  As fc;""")
rua_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Definindo o mapa e suas camadas
m = folium.Map(location = [-25.44,-49.24],
               zoom_start = 15,
               tiles = 'Cartodb Positron',
               attr = '© contribuidores do OpenStreetMap (CC BY-SA 2.0)')

ruas_selec_emoc = folium.GeoJson(
    rua_geojson[2:len(rua_geojson)-2],
    name = 'Via escolhida',
    style_function = lambda feature: {
            'color': 'black',
            'weight': 2,
            },  
    overlay=True
    ).add_to(m)

# Grupos geral de camadas 
ft_grp1 = folium.FeatureGroup(name = 'Positivas', show = False)
ft_grp2 = folium.FeatureGroup(name = 'Negativas/Neutras', show = False)

VIADUTO DO CAPANEMA


In [ ]:
# Funcao que ira testar se o geojson do banco de dados veio com dados ou está vazio. Isto é, se a rua tem a emoçao consultada ou  nao
def If_gjson_is_None(geojson, icon_url):
  if geojson['features'] != None:
    for feature in geojson['features']:
      lon, lat = feature['geometry']['coordinates']
      icon = folium.features.CustomIcon(icon_url,
                                        icon_size=(20, 20))
      marker = folium.Marker([lat, lon], icon=icon)
      return marker
  else:
    # Retorna um marker fora da área de estudo. Ideia mudar a simbologia para ficar transparente
    return folium.Marker([0,0])

In [ ]:
# Valor escolhido no combo box
nm_rua = comboRuas.value

# Consulta no banco da geometria da rua escolhida
cursor.execute("""CREATE OR REPLACE VIEW rua_selec AS SELECT ruas_emoc_wgs.* FROM ruas_emoc_wgs WHERE nmvia = '%s'; """ %nm_rua)
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
 FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l )) As properties
 FROM rua_selec As lg   ) As f )  As fc;""")
rua_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Processo para simbolizar as emocoes. Sera necessário fazer a consulta para todas as emocoes, e depois na hora de simbolizar testar quais dados sao diferentes de nulo
# --- ANIMACAO --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.2) As geom_pts FROM rua_selec WHERE animacao >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua1 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua1 = json.loads(lnstopts_geojson_rua1[2:len(lnstopts_geojson_rua1)-2])
# Definindo a URL do emoji 
icon_url1 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/12.png'
# Grupo da emocao para o layer control
grp1 = folium.plugins.FeatureGroupSubGroup(ft_grp1, 'Animacao')

# --- ALEGRIA --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM rua_selec WHERE alegria >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua2 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua2 = json.loads(lnstopts_geojson_rua2[2:len(lnstopts_geojson_rua2)-2])
# Definindo a URL do emoji 
icon_url2 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/10.png'
# Grupo da emocao para o layer control
grp2 = folium.plugins.FeatureGroupSubGroup(ft_grp1, 'Alegria')

# --- CANSACO --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.5) As geom_pts FROM rua_selec WHERE cansaco >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua3 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua3 = json.loads(lnstopts_geojson_rua3[2:len(lnstopts_geojson_rua3)-2])
# Definindo a URL do emoji 
icon_url3 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/2.png'
# Grupo da emocao para o layer control
grp3 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Cansaco')

# --- BARULHO --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.2) As geom_pts FROM rua_selec WHERE p_barulho >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua4 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua4 = json.loads(lnstopts_geojson_rua4[2:len(lnstopts_geojson_rua4)-2])
# Definindo a URL do emoji 
icon_url4 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/21.png'
# Grupo da emocao para o layer control
grp4 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Barulho')

# --- ANSIEDADE --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.4) As geom_pts FROM rua_selec WHERE ansiedade >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua5 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua5 = json.loads(lnstopts_geojson_rua5[2:len(lnstopts_geojson_rua5)-2])
# Definindo a URL do emoji 
icon_url5 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/4.png'
# Grupo da emocao para o layer control
grp5 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Ansiedade')

# --- RAIVA/ODIO --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.3) As geom_pts FROM rua_selec WHERE raiva_odio >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua6 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua6 = json.loads(lnstopts_geojson_rua6[2:len(lnstopts_geojson_rua6)-2])
# Definindo a URL do emoji 
icon_url6 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/1.png'
# Grupo da emocao para o layer control
grp6 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Raiva/Odio')

# --- TEDIO --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.6) As geom_pts FROM rua_selec WHERE tedio >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua7 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua7 = json.loads(lnstopts_geojson_rua7[2:len(lnstopts_geojson_rua7)-2])
# Definindo a URL do emoji 
icon_url7 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/3.png'
# Grupo da emocao para o layer control
grp7 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Tedio')

# --- TRANQUILIDADE --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM rua_selec WHERE tranquilid >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua8 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua8 = json.loads(lnstopts_geojson_rua8[2:len(lnstopts_geojson_rua8)-2])
# Definindo a URL do emoji 
icon_url8 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/5.png'
# Grupo da emocao para o layer control
grp8 = folium.plugins.FeatureGroupSubGroup(ft_grp1, 'Tranquilidade')

# --- GRATIDAO --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.3) As geom_pts FROM rua_selec WHERE gratidao >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua9 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua9 = json.loads(lnstopts_geojson_rua9[2:len(lnstopts_geojson_rua9)-2])
# Definindo a URL do emoji 
icon_url9 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/6.png'
# Grupo da emocao para o layer control
grp9 = folium.plugins.FeatureGroupSubGroup(ft_grp1, 'Gratidao')

# --- ADMIRACAO --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.5) As geom_pts FROM rua_selec WHERE admiracao >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua10 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua10 = json.loads(lnstopts_geojson_rua10[2:len(lnstopts_geojson_rua10)-2])
# Definindo a URL do emoji 
icon_url10 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/7.png'
# Grupo da emocao para o layer control
grp10 = folium.plugins.FeatureGroupSubGroup(ft_grp1, 'Admiracao')

# --- INTROSPECCAO --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM rua_selec WHERE introspec >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua11 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua11 = json.loads(lnstopts_geojson_rua11[2:len(lnstopts_geojson_rua11)-2])
# Definindo a URL do emoji 
icon_url11 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/8.png'
# Grupo da emocao para o layer control
grp11 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Introspeccao')

# --- NOJO --- #
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.8) As geom_pts FROM rua_selec WHERE nojo >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts As lg   ) As f )  As fc;""")
lnstopts_geojson_rua12 = json.dumps(cursor.fetchall())
conn.commit()
# Harmonizando o geojson
gjson_lnstopts_rua12 = json.loads(lnstopts_geojson_rua12[2:len(lnstopts_geojson_rua12)-2])
# Definindo a URL do emoji 
icon_url12 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/9.png'
# Grupo da emocao para o layer control
grp12 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Nojo')

In [ ]:
grp1.add_child(If_gjson_is_None(gjson_lnstopts_rua1, icon_url1))
grp2.add_child(If_gjson_is_None(gjson_lnstopts_rua2, icon_url2))
grp3.add_child(If_gjson_is_None(gjson_lnstopts_rua3, icon_url3))
grp4.add_child(If_gjson_is_None(gjson_lnstopts_rua4, icon_url4))
grp5.add_child(If_gjson_is_None(gjson_lnstopts_rua5, icon_url5))
grp6.add_child(If_gjson_is_None(gjson_lnstopts_rua6, icon_url6))
grp7.add_child(If_gjson_is_None(gjson_lnstopts_rua7, icon_url7))
grp8.add_child(If_gjson_is_None(gjson_lnstopts_rua8, icon_url8))
grp9.add_child(If_gjson_is_None(gjson_lnstopts_rua9, icon_url9))
grp10.add_child(If_gjson_is_None(gjson_lnstopts_rua10, icon_url10))
grp11.add_child(If_gjson_is_None(gjson_lnstopts_rua11, icon_url11))
grp12.add_child(If_gjson_is_None(gjson_lnstopts_rua12, icon_url12))

# Adicionando ao mapa os group layer
m.add_child(ft_grp1)
m.add_child(grp1)
m.add_child(grp2)
m.add_child(grp8)
m.add_child(grp9)
m.add_child(grp10)
m.add_child(ft_grp2)
m.add_child(grp3)
m.add_child(grp4)
m.add_child(grp5)
m.add_child(grp6)
m.add_child(grp7)
m.add_child(grp11)
m.add_child(grp12)

folium.LayerControl(
    collapsed = False
).add_to(m)

m

## Mapa 3 - Faça uma rota e descubra quais emoções foram atribuídas as ruas em que a rota passou

In [ ]:
# Biblioteca para as rotas
!pip install openrouteservice ortools

In [ ]:
# Entrada dos endereços para geocodificação
endereco = widgets.Text(
    placeholder='Digite o endereco de partida',
    disabled=False
)

endereco1 = widgets.Text(
    placeholder='Digite o endereco de chegada',
    disabled=False
)

# Layout dos botões
AppLayout(left_sidebar = VBox([Label("Ponto de partida:"), endereco]),
          right_sidebar = VBox([Label("Ponto de chegada:"),endereco1]),
          pane_widths=['500px', 1, 1],
          pane_heights=['500px', 4, 1],
          grid_gap="30px")

# Rotas sugeridas
# Rua Carlos Pradi, 163
# Rua Amoroso Costa, 100

QXBwTGF5b3V0KGNoaWxkcmVuPShWQm94KGNoaWxkcmVuPShMYWJlbCh2YWx1ZT11J1BvbnRvIGRlIGNoZWdhZGE6JyksIFRleHQodmFsdWU9dScnLCBwbGFjZWhvbGRlcj11J0RpZ2l0ZSBvIGXigKY=


In [ ]:
from geopy.geocoders import Nominatim
from openrouteservice import client

# estilo da rota
style = {'color': '#969696'}

# API do openroute services
api_key = '5b3ce3597851110001cf6248997aca5cd66d41c2aa3ad5f61948b94e'
clnt = client.Client(key=api_key) # Create client with api key

# Pede e geocodifica os dois endereços
geolocator = Nominatim(user_agent = 'camaragabriele@gmail.com')
location = geolocator.geocode(endereco.value)
location1 = geolocator.geocode(endereco1.value)

# Coloca os pois pontos no mapa
m = folium.Map (location= [location.latitude, location.longitude],
                zoom_start = 14,
                tiles = 'Cartodb Positron',
                min_zoom = 15,
                attr = '© contribuidores do OpenStreetMap (CC BY-SA 2.0)')
folium. Marker (location=[location.latitude, location.longitude],
                popup='Ponto de partida: '+endereco.value,
                icon=folium.Icon(color='blue',icon='star')).add_to(m)
folium. Marker (location=[location1.latitude, location1.longitude], 
                popup='Ponto de chegada: '+endereco1.value,
                icon=folium.Icon(color='green',icon='star')).add_to(m)

# Requisição da Rota                 
request_params = {'coordinates': [[location.longitude, location.latitude],
                                 [location1.longitude, location1.latitude]],
                'format_out': 'geojson',
                'profile': 'driving-car',
                'preference': 'shortest',
                'instructions': 'false',}
route_normal = clnt.directions(**request_params)

# Adiciona a rota no mapa
rota = folium.features.GeoJson(data=route_normal,
                        name='Rota entre os pontos',
                        style_function=lambda x: style,
                        overlay=True).add_to(m)                 

dmb_gjson = json.loads(json.dumps(route_normal))

cursor.execute("""CREATE TABLE rota(id integer primary key); SELECT AddGeometryColumn ('public','rota','geom', 4326,'LINESTRING', 2);""")
conn.commit()

for feature in dmb_gjson['features']:
  geom = json.dumps(feature['geometry'])
  id = 1
  cursor.execute("""INSERT INTO rota (id, geom) VALUES (%s, ST_SetSRID(ST_GeomFromGeoJSON('%s'), 4326) )""" %(id, geom))
  conn.commit()

# Geometria de linha
cursor.execute("""CREATE OR REPLACE VIEW ruas_emoc_rotas AS SELECT ruas_emoc_wgs.* FROM rota, ruas_emoc_wgs WHERE ST_DWithin(ST_Transform(rota.geom, 31982), ST_Transform(ruas_emoc_wgs.geom_wgs, 31982), 50);""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_wgs)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM ruas_emoc_rotas As lg ) As f )  As fc;""")
rota_emoc_geojson = json.dumps(cursor.fetchall())
conn.commit()

# Geometria de ponto - Processo manual para simbolizar todas as emocoes
# ANIMACAO
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts1 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE animacao >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts1 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua1 = json.dumps(cursor.fetchall())
conn.commit()

# ALEGRIA
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts2 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE alegria >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts2 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua2 = json.dumps(cursor.fetchall())
conn.commit()

# CANSACO
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts3 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE cansaco >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts3 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua3 = json.dumps(cursor.fetchall())
conn.commit()

# BARULHO
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts4 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE p_barulho >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts4 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua4 = json.dumps(cursor.fetchall())
conn.commit()

# ANSIEDADE
cursor.execute("""CREATE OR REPLACE VIEW rua_selec_pts5 AS SELECT *, ST_LineInterpolatePoint(st_linemerge(geom_wgs), 0.9) As geom_pts FROM ruas_emoc_rotas WHERE ansiedade >= 1;""")
cursor.execute("""SELECT row_to_json(fc) FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
FROM (SELECT 'Feature' As type, ST_AsGeoJSON(lg.geom_pts)::json As geometry, row_to_json((SELECT l FROM (SELECT gid) As l)) As properties
FROM rua_selec_pts5 As lg   ) As f )  As fc;""")
lnstopts_geojson_rua5 = json.dumps(cursor.fetchall())
conn.commit()

# Camada das linhas
ruas_emoc_rota = folium.GeoJson(
    rota_emoc_geojson[2:len(rota_emoc_geojson)-2],
    name = 'Rua pertencente a rota',
    style_function = lambda feature: {
            'color': 'black',
            'weight': 1,
            },  
    overlay=True
    ).add_to(m)

# Grupos de camadas
ft_grp1 = folium.FeatureGroup(name = 'Positivas', show = False)
ft_grp2 = folium.FeatureGroup(name = 'Negativas', show = False)
grp1 = folium.plugins.FeatureGroupSubGroup(ft_grp1, 'Animacao')
grp2 = folium.plugins.FeatureGroupSubGroup(ft_grp1, 'Alegria')
grp3 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Cansaco')
grp4 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Barulho')
grp5 = folium.plugins.FeatureGroupSubGroup(ft_grp2, 'Ansiedade')

# Camada dos pontos sobre as linhas
gjson_lnstopts_rua1 = json.loads(lnstopts_geojson_rua1[2:len(lnstopts_geojson_rua1)-2])
gjson_lnstopts_rua2 = json.loads(lnstopts_geojson_rua2[2:len(lnstopts_geojson_rua2)-2])
gjson_lnstopts_rua3 = json.loads(lnstopts_geojson_rua3[2:len(lnstopts_geojson_rua3)-2])
gjson_lnstopts_rua4 = json.loads(lnstopts_geojson_rua4[2:len(lnstopts_geojson_rua4)-2])
gjson_lnstopts_rua5 = json.loads(lnstopts_geojson_rua5[2:len(lnstopts_geojson_rua5)-2])

# --- ANIMACAO
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'animacao'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url1 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua1['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url1,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp1.add_child(marker)

# --- ALEGRIA
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'alegria'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url2 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua2['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url2,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp2.add_child(marker)

# --- CANSACO
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'cansaco'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url3 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua3['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url3,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp3.add_child(marker)

# --- BARULHO
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'p_barulho'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url4 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua4['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url4,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp4.add_child(marker)

# --- ANSIEDADE
# Consulta do id do emoji da emocao escolhida
cursor.execute("""SELECT emoc_id FROM emocoes WHERE cod_emoc_ruas = 'ansiedade'; """)
id_emoji = cursor.fetchall()
conn.commit()
icon_url5 = 'https://raw.githubusercontent.com/GabrieleCamara/emotional_maps/master/emojis/%s.png' %id_emoji[0][0]

# Estilizando os pontos sobre as linhas
for feature in gjson_lnstopts_rua5['features']:
  lon, lat = feature['geometry']['coordinates']
  icon = folium.features.CustomIcon(icon_url5,
                                    icon_size=(20, 20))
  marker = folium.Marker([lat, lon], icon=icon)
  grp5.add_child(marker)

# Adicionando ao mapa os group layer
m.add_child(ft_grp1)
m.add_child(grp1)
m.add_child(grp2)
m.add_child(ft_grp2)
m.add_child(grp3)
m.add_child(grp4)
m.add_child(grp5)

folium.LayerControl(
).add_to(m)

m

In [ ]:
# Executar essa celula antes de fazer uma nova rota!
cursor.execute(""" DROP TABLE IF EXISTS rota CASCADE; """)
conn.commit()